# GAN implementation by tensorflow

In [ ]:
import tensorflow as tf
import os
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

# utility


In [ ]:
def save_metrics(metrics, epoch=None):
    # make directory if there is not
    path = "metrics_DC"
    if not os.path.isdir(path):
        os.makedirs(path)

    plt.figure(figsize=(10,8))
    plt.plot(metrics["dis_loss"], label="discriminative loss", color="b")
    plt.legend()
    plt.savefig(os.path.join(path, "dloss" + str(epoch) + ".png"))
    plt.close()

    plt.figure(figsize=(10,8))
    plt.plot(metrics["gen_loss"], label="generative loss", color="r")
    plt.legend()
    plt.savefig(os.path.join(path, "g_loss" + str(epoch) + ".png"))
    plt.close()

    plt.figure(figsize=(10,8))
    plt.plot(metrics["gen_loss"], label="generative loss", color="r")
    plt.plot(metrics["dis_loss"], label="discriminative loss", color="b")
    plt.legend()
    plt.savefig(os.path.join(path, "both_loss" + str(epoch) + ".png"))
    plt.close()

In [ ]:
# save generated images
def save_imgs(images, plot_dim=(5,12), size=(12,5), epoch=None):
    # make directory if there is not
    path = "generated_figures_DC"
    if not os.path.isdir(path):
        os.makedirs(path)

    examples = plot_dim[0]*plot_dim[1]

    # 表示
    fig = plt.figure(figsize=size)
    for i in range(examples):
        plt.subplot(plot_dim[0], plot_dim[1], i+1)
        img = images[i, :]
        img = img.reshape((256, 256, 3))
        plt.tight_layout()
        plt.imshow(img)
        plt.axis("off")
    plt.subplots_adjust(wspace=0.1, hspace=0.1)
    plt.savefig(os.path.join(path, str(epoch) + ".png"))
    plt.close()


# モデル

In [ ]:
class Generator:
    def __init__(self):
        self.reuse = False
        self.z_dim = 100
        self.s_size = 2
        self.depths = [1024, 512, 256, 128,  64, 32, 3]
        
    def __call__(self, inputs, training=False):
        with tf.variable_scope('g', reuse=self.reuse):
            # reshape from inputs
            outputs = tf.reshape(inputs, [-1, self.z_dim])
            
            with tf.variable_scope('reshape'):
                outputs = tf.layers.dense(outputs, self.depths[0] * self.s_size * self.s_size) 
                outputs = tf.reshape(outputs, [-1, self.s_size, self.s_size, self.depths[0]]) # 2x2
                outputs = tf.nn.relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')

            with tf.variable_scope('deconv0'):
                outputs = tf.layers.conv2d_transpose(outputs, self.depths[0], [4, 4], strides=(2, 2), padding='SAME', kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02)) # 4x4
                outputs = tf.nn.relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')

            with tf.variable_scope('deconv1'):
                outputs = tf.layers.conv2d_transpose(outputs, self.depths[1], [4, 4], strides=(2, 2), padding='SAME', kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02)) # 8x8
                outputs = tf.nn.relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')

            with tf.variable_scope('deconv2'):
                outputs = tf.layers.conv2d_transpose(outputs, self.depths[2], [4, 4], strides=(2, 2), padding='SAME', kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02)) # 16x16
                outputs = tf.nn.relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')

            with tf.variable_scope('deconv3'):
                outputs = tf.layers.conv2d_transpose(outputs, self.depths[3], [4, 4], strides=(2, 2), padding='SAME', kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02)) # 32x32
                outputs = tf.nn.relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')
            with tf.variable_scope('deconv4'):
                outputs = tf.layers.conv2d_transpose(outputs, self.depths[4], [4, 4], strides=(2, 2), padding='SAME', kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02)) # 64x64
                outputs = tf.nn.relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')
            with tf.variable_scope('deconv5'):
                outputs = tf.layers.conv2d_transpose(outputs, self.depths[5], [4, 4], strides=(2, 2), padding='SAME', kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02)) # 128x128
                outputs = tf.nn.relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')
            with tf.variable_scope('deconv6'):
                outputs = tf.layers.conv2d_transpose(outputs, self.depths[6], [4, 4], strides=(2, 2), padding='SAME', kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02)) # 256x256
                outputs = tf.layers.batch_normalization(outputs)
            # output images
            with tf.variable_scope('tanh'):
                outputs = tf.nn.tanh(outputs)
                #outputs = tf.sigmoid(outputs)
                outputs = tf.reshape(outputs, [-1, 256, 256, 3])

        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='g')
        return outputs

In [ ]:
class Discriminator:
    def __init__(self):
        self.reuse = False
        self.X_dim = 256*256*3
        self.s_size = 4
        self.depths = [32, 64, 128, 256, 512, 1024, 1]

    def __call__(self, inputs, training=False, name=''):
        def leaky_relu(x, leak=0.2, name='outputs'):
            return tf.maximum(x, x * leak, name=name)

        with tf.name_scope('d' + name), tf.variable_scope('d', reuse=self.reuse):
            # convolution x 4
            outputs = tf.reshape(inputs, [-1, 256, 256, 3])
            with tf.variable_scope('conv0'):
                outputs = tf.layers.conv2d(outputs, self.depths[0], [4, 4], strides=(2, 2), padding='SAME', kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02)) # 128x128
                outputs = leaky_relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')
            with tf.variable_scope('conv1'):
                outputs = tf.layers.conv2d(outputs, self.depths[1], [4, 4], strides=(2, 2), padding='SAME', kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02)) # 64x64
                outputs = leaky_relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')
            with tf.variable_scope('conv2'):
                outputs = tf.layers.conv2d(outputs, self.depths[2], [4, 4], strides=(2, 2), padding='SAME', kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02)) # 32x32
                outputs = leaky_relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')
            with tf.variable_scope('conv3'):
                outputs = tf.layers.conv2d(outputs, self.depths[3], [4, 4], strides=(2, 2), padding='SAME', kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02)) # 16x16
                outputs = leaky_relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')
            with tf.variable_scope('conv4'):
                outputs = tf.layers.conv2d(outputs, self.depths[4], [4, 4], strides=(2, 2), padding='SAME', kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02)) # 8x8
                outputs = leaky_relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')
            with tf.variable_scope('conv5'):
                outputs = tf.layers.conv2d(outputs, self.depths[5], [4, 4], strides=(2, 2), padding='SAME', kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02)) # 4x4
                outputs = leaky_relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')
            with tf.variable_scope('conv6'):
                outputs = tf.layers.conv2d(outputs, self.depths[6], [4, 4], strides=(2, 2), padding='SAME', kernel_initializer=tf.truncated_normal_initializer(0.0, 0.02)) # 2x2x1
                outputs = leaky_relu(tf.layers.batch_normalization(outputs, training=training), name='outputs')
            with tf.variable_scope('classify'):
                print(outputs.get_shape())
                outputs = tf.reshape(outputs, [-1, 2*2*1])
                outputs = tf.layers.dense(outputs, 1, name='outputs')
                print(outputs.get_shape())

        self.reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='d')
        return outputs

In [ ]:
class DCGAN:
    def __init__(self, batch_size=2):
        self.batch_size = batch_size
        self.g = Generator()
        self.d = Discriminator()
        self.z_dim = 100

    def d_loss_calc(self, g_outputs, t_outputs, gradient_penalty):
        lambda_ = 10

        d_loss_train = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=t_outputs, labels=tf.ones_like(t_outputs)))
        d_loss_gen = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=g_outputs, labels=tf.zeros_like(g_outputs)))
        loss = d_loss_train + d_loss_gen
        loss = loss + lambda_*gradient_penalty
        return loss

    def g_loss_calc(self, g_outputs):
        loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=g_outputs, labels=tf.ones_like(g_outputs)))
        return loss

    def loss(self, traindata, perturbed_minibatch):
        z = tf.random_uniform([self.batch_size, self.z_dim], minval=-1.0, maxval=1.0)
        generated = self.g(z, training=True)
        g_outputs = self.d(generated, training=True, name='g')
        t_outputs = self.d(traindata, training=True, name='t')
        
        alpha = tf.random_uniform(shape=[self.batch_size,1], minval=0., maxval=1.)

        diff = perturbed_minibatch - traindata
        interpolates = traindata + (alpha*diff)
        gradients = tf.gradients(self.d(interpolates), [interpolates])[0]
        slopes = tf.sqrt(tf.reduce_sum(tf.square(gradients), reduction_indices=[1]))
        gradient_penalty = tf.reduce_mean((slopes-1.)**2)

        # add each losses to collection
        tf.add_to_collection(
            'g_losses',
            self.g_loss_calc(g_outputs))

        tf.add_to_collection(
            'd_losses',
            self.d_loss_calc(g_outputs, t_outputs, gradient_penalty)
        )

        return {
            self.g: tf.add_n(tf.get_collection('g_losses'), name='total_g_loss'),
            self.d: tf.add_n(tf.get_collection('d_losses'), name='total_d_loss'),
        }

    def d_train(self, losses, learning_rate=1e-4, beta1=0.5, beta2=0.9):
        d_opt = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=beta1, beta2=beta2)
        d_opt_op = d_opt.minimize(losses[self.d], var_list=self.d.variables)
        return d_opt_op

    def g_train(self, losses, learning_rate=1e-4, beta1=0.5, beta2=0.9):
        g_opt = tf.train.AdamOptimizer(learning_rate=learning_rate, beta1=beta1, beta2=beta2)
        g_opt_op = g_opt.minimize(losses[self.g], var_list=self.g.variables)
        return g_opt_op

    def sample_images(self, row=5, col=12, inputs=None, epoch=None):
        images = self.g(inputs, training=True)
        return images

In [ ]:
import numpy as np

#X_train = np.load("irasutoya_face_1813x256x256x3_jpg.npy")
#X_train = X_train/255
X_dim = 256*256*3
z_dim = 100
batch_size = 128
epochs = 500000
display_epoch = 1000
param_save_epoch = 5000
loss = {"dis_loss":[], "gen_loss":[]}

In [ ]:
p_noise = tf.placeholder(tf.float32, [None, z_dim])
noise_check = np.random.uniform(-1, 1, size=[60, z_dim]).astype(np.float32)

X = tf.placeholder(tf.float32, [None, X_dim])
X_p = tf.placeholder(tf.float32, [None, X_dim])

dcgan = DCGAN()
losses = dcgan.loss(traindata=X, perturbed_minibatch=X_p)
d_train_op = dcgan.d_train(losses)
g_train_op = dcgan.g_train(losses)

saver = tf.train.Saver()
#%debug
config = tf.ConfigProto(
    gpu_options=tf.GPUOptions(
        visible_device_list="0"
    )
)

with tf.Session(config=config) as sess:

    sess.run(tf.global_variables_initializer())
    for epoch in range(epochs):
        if epoch % display_epoch == 0:
            imgs_gen = dcgan.sample_images(inputs=noise_check).eval() 
            imgs_gen += 0.5
            print("saving images")
            save_imgs(imgs_gen, epoch=epoch)
        for _ in range(10):
            # X_mb, Y_mbを収集
            def extractXimg(path, batch_size):
                imgs = os.listdir(path)
                rand_id = np.random.randint(0, len(imgs), size=batch_size)
                X_mb = np.zeros((1, 512, 512, 3))

                for i in range(batch_size):
                    img = Image.open(path+"/"+imgs[rand_id[i]])

                    width, height = img.size
                    square_size = min(img.size)

                    if width > height:
                        top = 0
                        bottom = square_size
                        left = (width - square_size) / 2
                        right = left + square_size
                        box = (left, top, right, bottom)
                    else:
                        left = 0
                        right = square_size
                        top = (height - square_size) / 2
                        bottom = top + square_size
                        box = (left, top, right, bottom)

                    img = img.crop(box)
                    thumbnail_size = (1024, 1024)
                    img = img.resize(thumbnail_size, Image.ANTIALIAS)
                    #img.thumbnail(thumbnail_size)
                    #print(img.size)
                    img_np = np.asarray(img)
                    X_mb = np.vstack((X_mb, img_np[np.newaxis, :]))
                    img.close()
                X_mb = X_mb[1:,:]
                return X_mb
                

            # 訓練データを抜粋
            #rand_index = np.random.randint(0, X_train.shape[0], size=batch_size)

            X_mb = extractXimg("./256x256x3resized_faceimgs", batch_size)
            X_mb = -0.5 + X_mb/255

            #X_mb = X_train[rand_index, :].astype(np.float32)
            X_mb = X_mb.reshape([-1, X_dim])

            # x + delta vector
            X_mb_p = X_mb + 0.5*X_mb.std()*np.random.random(X_mb.shape)

            _, d_loss_value = sess.run([d_train_op, losses[dcgan.d]], feed_dict={X: X_mb, X_p:X_mb_p})
        _, g_loss_value, = sess.run([g_train_op, losses[dcgan.g]], feed_dict={X: X_mb, X_p:X_mb_p})

        # 結果をappend
        loss["dis_loss"].append(d_loss_value)
        loss["gen_loss"].append(g_loss_value)
        print("epoch:" + str(epoch))
        # グラフの描画（余裕があったら）
        if epoch % display_epoch == 0:
            save_metrics(loss, epoch)

        if epoch % param_save_epoch == 0:
            path = "model"
            if not os.path.isdir(path):
                os.makedirs(path)
            saver.save(sess, path + "/dcgan_model" + str(epoch) + ".ckpt")